# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /home/ian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import nltk
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.externals import joblib


import pickle


/home/ian/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_data.db')
df = pd.read_sql_table(table_name = 'message_data', con=engine)

X = df.message
Y = df[['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity',
'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report', ]]

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [43]:
y_pred.shape[1]

36

In [6]:
#testing what actual predictions look like

res = pipeline.predict(['Help there is a flood'])
res

labels = ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity',
'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']

for index, val in enumerate(res[0]):
    print(labels[index] + ":" + str(val))

related:1
request:0
offer:0
aid_related:0
medical_help:0
medical_products:0
search_and_rescue:0
security:0
military:0
child_alone:0
water:0
food:0
shelter:0
clothing:0
money:0
missing_people:0
refugees:0
death:0
other_aid:0
infrastructure_related:0
transport:0
buildings:0
electricity:0
tools:0
hospitals:0
shops:0
aid_centers:0
other_infrastructure:0
weather_related:1
floods:0
storm:0
fire:0
earthquake:0
cold:0
other_weather:0
direct_report:0


In [42]:
y_test.shape

(6554, 36)

In [7]:
# evaluate all steps on test set
y_pred = pipeline.predict(X_test)

for col in range(y_pred.shape[1]):
    print("Class: " + str(col))
    print(classification_report(y_test.iloc[:, col], y_pred[:, col]))

Class: 0
             precision    recall  f1-score   support

          0       0.60      0.36      0.45      1514
          1       0.83      0.93      0.88      5040

avg / total       0.78      0.80      0.78      6554

Class: 1
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5403
          1       0.84      0.37      0.52      1151

avg / total       0.87      0.88      0.86      6554

Class: 2
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

Class: 3
             precision    recall  f1-score   support

          0       0.73      0.88      0.80      3866
          1       0.76      0.53      0.62      2688

avg / total       0.74      0.74      0.73      6554

Class: 4
             precision    recall  f1-score   support

          0       0.93      1.00      0.9

/home/ian/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

Note: I have found the runtime of a useful-sized grid to be unmanagable given the real-life time I have to work on this project.  For the purposes of learning how to createa ML pipeline, I have cut the grid search size down significantly.

In [10]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__n_estimators': [100, 200],
        'clf__estimator__min_samples_split': [2, 3],
        }


cv =  GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=-1)
cv.fit(X_train, y_train)

#best_clf = cv.best_estimator_



''' EXAMPLE FROM CASE STUDY
    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    
    
'''

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__ngram_range=(1, 1) 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__ngram_range=(1, 1) 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__ngram_range=(1, 1) 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__ngram_range=(1, 1), total= 5.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__ngram_range=(1, 1), total= 6.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__ngram_range

[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed: 97.0min finished


" EXAMPLE FROM CASE STUDY\n    parameters = {\n        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),\n        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),\n        'features__text_pipeline__vect__max_features': (None, 5000, 10000),\n        'features__text_pipeline__tfidf__use_idf': (True, False),\n        'clf__n_estimators': [50, 100, 200],\n        'clf__min_samples_split': [2, 3, 4],\n        'features__transformer_weights': (\n            {'text_pipeline': 1, 'starting_verb': 0.5},\n            {'text_pipeline': 0.5, 'starting_verb': 1},\n            {'text_pipeline': 0.8, 'starting_verb': 1},\n        )\n    }\n\n    cv = GridSearchCV(pipeline, param_grid=parameters)\n    \n    \n    \n"

In [21]:
cv.predict(["Help I need food"])

array([[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

Accuracy = TP+TN/TP+FP+FN+TN

Precision = TP/TP+FP

Recall = TP/TP+FN

F1 = 2 * (precision * recall) / (precision + recall)



In [10]:
def display_results(cv, y_test, y_pred):
    labels = np.unique(y_pred)
    #confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()
    precision = (y_pred == y_test).sum() / len(y_pred)
    recall = (y_pred == y_test).sum() / y_pred.sum()
    
    print("Labels:", labels)

    print("Accuracy:")
    print(accuracy)
    
    print("Precision:")
    print(precision)
    
    print("Recall:")
    print(recall)
    
    f1 = 2 * (precision * recall) / (precision + recall)
    
    print("F1:")
    print(f1)
    
    
    
    #print("\nBest Parameters:", cv.best_params)

    
display_results(cv, y_test, y_pred)

Labels: [0 1]
Accuracy:
related                   0.793866
request                   0.879463
offer                     0.994965
aid_related               0.719866
medical_help              0.921269
medical_products          0.955600
search_and_rescue         0.970247
security                  0.981843
military                  0.969027
child_alone               1.000000
water                     0.953158
food                      0.929204
shelter                   0.928135
clothing                  0.986421
money                     0.976198
missing_people            0.989319
refugees                  0.967806
death                     0.958346
other_aid                 0.866646
infrastructure_related    0.931034
transport                 0.953616
buildings                 0.951022
electricity               0.978639
tools                     0.993744
hospitals                 0.990082
shops                     0.993744
aid_centers               0.989014
other_infrastructure      0.951

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [23]:
ada = AdaBoostClassifier()

pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator = ada)),
])

# train classifier
pipeline2.fit(X_train, y_train)

# evaluate all steps on test set
y_pred2 = pipeline2.predict(X_test)



for col in range(y_pred2.shape[1]):
    print("Class: " + str(col))
    print(classification_report(y_test.iloc[:, col], y_pred2[:, col]))

print('OPTIMIZED ADABOOST MODEL')

parameters2 = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'clf__n_estimators': [50, 100, 200],
        'clf__learning_rate': [1.0, 1.2, 1.4, 1.6, 1.8] 
        }


cv2 =  GridSearchCV(pipeline2, param_grid=parameters2)
#best_clf2 = cv2.best_estimator_


display_results(cv2, y_test, y_pred2)

Class: 0
             precision    recall  f1-score   support

          0       0.58      0.24      0.34      1534
          1       0.80      0.94      0.86      4976
          2       0.27      0.14      0.18        44

avg / total       0.74      0.77      0.74      6554

Class: 1
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5445
          1       0.79      0.54      0.64      1109

avg / total       0.89      0.90      0.89      6554

Class: 2
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6522
          1       0.00      0.00      0.00        32

avg / total       0.99      0.99      0.99      6554

Class: 3
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3820
          1       0.76      0.61      0.67      2734

avg / total       0.76      0.75      0.75      6554

Class: 4
             precision    recall  f1-score

### 9. Export your model as a pickle file

In [11]:
pkl_filename = 'model.pkl'
pkl_outfile = open(pkl_filename,'wb')
joblib.dump(cv.best_estimator_, pkl_filename)
pkl_outfile.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.